In [1]:
import optkit as ok
import numpy as np
ok.set_backend(gpu=True, double=False)

optkit backend set to cpu64
Libraries for configuration gpu32 not found. Trying next configuration.
Libraries for configuration gpu64 not found. Trying next configuration.
Libraries for configuration cpu32 not found. Trying next configuration.
optkit backend set to cpu64


1

In [2]:
import radopt as ro
import IJOC_analyses
from IPython.display import display, Math, Markdown
import bokeh, bokeh.plotting, bokeh.io

In [3]:
bokeh.io.output_notebook()

Loading BokehJS ...

### load dose matrices, clustering maps

In [4]:
N_STRUCTURES = 16
N_TARGETS = 3 

N = N_BEAMS = 500
K = N_CLUSTERS = 50
M_s = VOX_PER_STRUCTURE = 100
M = N_STRUCTURES * M_s

replace the next cell with:
===============
- loading dose matrices
- loading beam clustering assignments (if pre-calculated)
- loading beam-clustered matrices (if pre-calculated)

or, load the dose matrices and
- calculate beam clustering assignments (as below)
- calculate beam-clustered matrices (as below).

------

**_In this example,_** we generate random dose matrices
$$A_s \in \mathbf{R}_+^{m_s \times n},\quad s=1,\ldots,N_s$$
for each structure `s`, with the values drawn 
from a normal distribution and thresholded to be nonnegative.

We generate a single cluster assignment matrix,
$$V \in \mathbf{R}^{k \times n}$$
which we use to form the beam-clustered dose matrices
$$A_{\mathcal{C}s} \in \mathbf{R}^{m_s \times k}$$ 
such that 
$$A_s \approx A_{\mathcal{C}s}V^T,\quad s=1,\ldots,N_s,$$
i.e., each structure's full dose matrix is approximated by the
its corresponding beam-clustered dose matrix when the columns
of the latter are upsampled according to the cluster assignments
encoded by `V`.

In [5]:
stdev = 0.1
def meanval(structure_id):
    return 1.0 if structure_id == 0 else (0.9 if structure_id < N_TARGETS else 0.1)
def random_dosemat(structure_id):
    return np.maximum(0, np.random.normal(meanval(structure_id), stdev, (VOX_PER_STRUCTURE, N_BEAMS)))

# dose matrices per structure with full # of beams
A_s = {k: random_dosemat(k) for k in range(N_STRUCTURES)}

# assemble into single matrix
A = np.vstack(A_s.values())

# run k-means on columns of assembled matrix (in implementation, on rows of transpose)
v_guess = np.sort(np.hstack((np.random.permutation(K), np.random.randint(0, K, N-K))))
_, v, _ = ok.api.Clustering().kmeans(A.T, K, v_guess)
cluster_map = ro.clustering.ClusterMap(N, v.max()+1, v)

# apply clustering from k-means to form beam-clustered matrices per structure
bclu_matrices = ro.clustering.downsample(cluster_map, A_s, transpose=True)
full_matrices = A_s

### build structures

In [6]:
names = {
    0: 'PTV 66Gy',
    1: 'PTV R Frontal + Neck 60Gy',
    2: 'PTV L Frontal 60Gy',
    3: 'Eye L',
    4: 'Lens L',
    5: 'Brainstem',
    6: 'Spinal Canal',
    7: 'Submandibular L',
    8: 'Cochlea R',
    9: 'Larynx',
    10: 'Oral Cavity',
    11: 'Brain',
    12: 'Optic Nerve L',
    13: 'Optic Chiasm',
    14: 'Cord',
    15: 'Body',
}
target_ids = list(range(N_TARGETS))
OAR_ids = list(range(N_TARGETS, N_STRUCTURES)) 
structure_order = target_ids + OAR_ids

In [7]:
sizes = {s: full_matrices[s].shape[0] for s in structure_order}  

structures = {
        sid: ro.structure.Target(names[sid], sizes[sid]) 
        for sid in target_ids}    

structures.update({
        sid: ro.structure.OAR(names[sid], sizes[sid]) 
        for sid in OAR_ids})

In [8]:
structures[0].objective.parameters['dose'] = 66.
structures[1].objective.parameters['dose'] = 60.
structures[2].objective.parameters['dose'] = 60.

### build clinical constraints

In [9]:
constraints = dict()
constraints['PTV 66Gy'] = (ro.constraint.D(95) > 66,)
constraints['PTV R Frontal + Neck 60Gy'] = (ro.constraint.D(95) > 60,)
constraints['PTV L Frontal 60Gy'] = (ro.constraint.D(95) > 60,)
#### These constraints used for experiments; unlikely to hold with random dose matrices & clusters
# constraints['Cord'] = (ro.constraint.D(1) < 50,) 
# constraints['Brainstem'] = (ro.constraint.D(1) < 54,)
# constraints['Optic Nerve L'] = (ro.constraint.D(1) < 54,)
# constraints['Optic Chiasm'] = (ro.constraint.D(1) < 55,)
# constraints['Spinal Canal'] = (ro.constraint.D(1) < 50,)
# # constraints['Parotid L'] = (ro.constraint.D('mean') < 20,)
# constraints['Larynx'] = (ro.constraint.D('mean') < 44, ro.constraint.D(27) < 55)
# constraints['Cochlea R'] = (ro.constraint.D('mean') < 45,)


### build clustered problem

In [10]:
problem = ro.compression.BeamClusteredProblem(
    structures=structures,
    A_full=full_matrices,
    A_bclu=bclu_matrices,
    cluster_map=cluster_map)
# problem.structures.update(structures)
# problem.A_full.update(full_matrices)
# problem.A_bclu.update(bclu_matrices)
# problem.cluster_map = cluster_map
problem.set_key_order(structure_order)
constraints = problem.rekey_dictionary_by_ids(constraints)

### build full problem

In [11]:
full_problem = problem.full_problem()

### cold start + pareto sweep, full problem

In [12]:
initial_weights = full_problem.current_weights()
weight_increments = {k: 2 for k in initial_weights}
weight_limits = {k: (0.3*iw, 3*iw) for k, iw in initial_weights.items()}

In [13]:
cfp = ro.pareto.ClinicallyFeasiblePareto()

In [14]:
solutions_full = cfp.explore_weights(
        full_problem, 
        initial_weights, 
        weight_increments, 
        weight_limits=weight_limits, 
        dose_constraints=constraints,
        verbose_pareto=False,
        verbose_constraints=False)

### cold start + pareto sweep, clustered problem(s)

In [15]:
solutions_clu = cfp.explore_weights(
        problem, 
        initial_weights, 
        weight_increments, 
        weight_limits=weight_limits, 
        dose_constraints=constraints,
        verbose_pareto=False,
        verbose_dual=False,
        verbose_constraints=False)

summary = ro.compression.CompressionAnalysis.analyze(solutions_clu, solutions_full)
# for other compressed approximations to the same full problem, can perform:
# >>> problem.A_vclu.update(new_compressed_matrix_dictionary)
# >>> problem.cluster_maps.update(new_cluster_map_dictionary)
# and then repeat this cell and any further analysis 

**_for additional compressed approximations_**
to the same full problem, can perform:
```python
problem.A_bclu.update(new_compressed_matrix_dictionary)
problem.cluster_map = update(new_cluster_map)
```
and then repeat the above cell and any further analysis 

In [16]:
summary_full = ro.compression.CompressionAnalysis.analyze(solutions_full)

## results

### tabular

In [17]:
display(Markdown(IJOC_analyses.bclu_table_cold(summary_full, summary)))


dimension |  primal (s) |  dual time (s) |  subopt (%) |  true error (%)
-|-|-|-|-
(313, 500) | 0.035 | -- | -- | -- 
(313, 50)| 0.016 | 0.009 | 21.0 | 3.0 

In [18]:
display(Math(IJOC_analyses.bclu_table_cold(summary_full, summary, latex=True)))

<IPython.core.display.Math object>

In [19]:
display(Markdown(IJOC_analyses.bclu_table_warm(summary_full, summary)))


dimension |  mean primal time (s) |  median primal time (s) |  mean dual time (s) |  median dual time (s) |  mean subopt (%) |  median subopt (%) |  mean true error (%) |  median true error (%) |  runs
-|-|-|-|-|-|-|-|-|-
(313, 500) | 0.007 | 0.006 | -- | -- | -- | -- | -- | -- | 38 
(313, 50)| 0.010 | 0.011 | 0.009 | 0.008 | 21.7 | 21.7 | 2.8 | 3.0 | 37 

### plots

In [20]:
y_full = ro.primal.A_mul_x(full_matrices, solutions_full['nominal']['beam_weights'])
y_bclu = ro.primal.A_mul_x(full_matrices, solutions_clu['nominal']['beam_weights'])

In [21]:
def dvh(voxel_doses):
    doses = np.hstack(([0], np.sort(voxel_doses)))
    percentiles = np.linspace(100, 0, len(doses + 1))#
    return (doses, percentiles)

In [22]:
plot = bokeh.plotting.figure()
plot.line(*dvh(y_full[0]), legend='full')
plot.line(*dvh(y_bclu[0]), line_dash=[5,2], line_width=2, legend='clustered')
plot.line(*dvh(y_full[1]), color='orange')
plot.line(*dvh(y_bclu[1]), color='orange', line_dash=[5,2], line_width=2)
plot.xaxis.axis_label = 'Dose (Gy)'
plot.yaxis.axis_label = 'Percentile'
plot.xaxis.axis_label_text_font_size = '1em'
plot.yaxis.axis_label_text_font_size = '1em'
plot.xaxis.major_label_text_font_size = '1em'
plot.yaxis.major_label_text_font_size = '1em'
plot.legend.label_text_font_size = '1em'
plot.legend.location = 'bottom_left'
plot.yaxis.bounds = [0,105]
plot.xaxis.bounds = [0, 100]
bokeh.plotting.show(plot)